In [3]:
# To install the mftool library
!pip install mftool==2.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
  Created wheel for mftool: filename=mftool-2.8-py3-none-any.whl size=116617 sha256=f857e9c753110d40687b40dbcfa4a353613de523c6137c6f4a68de26ecde83fe
  Stored in directory: /root/.cache/pip/wheels/b6/b3/af/5751d41db5452227e221aaca7f1ec823caa51ae7d40fb9dbde
Successfully built mftool


In [6]:
# https://medium.com/@TejasEkawade/getting-and-analyzing-mutual-funds-in-python-c2d0feb09881

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mftool import Mftool

mf = Mftool()
# result = mf.get_available_schemes('hdfc')
# for scheme_code, scheme in result.items():
#     print(scheme_code, scheme)

# Mutual Funds Analysis — Implementation
# Part 1: Tabular Representation
# Let’s pull data for multiple funds and save them as dataframes. These data frames will be saved in a dictionary.

def fetch_mutual_fund_data(mutual_fund_code):
    mf = Mftool()

    df = (mf.get_scheme_historical_nav(mutual_fund_code,as_Dataframe=True).reset_index()\
          .assign(nav=lambda x: x['nav'].astype(float),
                 date=lambda x: pd.to_datetime(x['date'], format='%d-%m-%Y'))\
          .sort_values('date')\
          .reset_index(drop=True)\
         )

    return df

# Fetch data for these funds
mutual_funds = {
    '125307': 'PGIM India Midcap Opportunities Fund - Direct Plan - Growth Option',
    '118989': 'HDFC Mid-Cap Opportunities Fund - Growth Option - Direct Plan',
    '127039': 'Motilal Oswal Midcap Fund-Regular Plan-Growth Option',
    '120726': 'UTI Mid Cap Fund-Growth Option- Direct'

}

mf_data_dict = dict()
for mutual_fund_code, mutual_fund_desc in mutual_funds.items():
    print(mutual_fund_desc)
    mf_data_dict[mutual_fund_desc] = fetch_mutual_fund_data(mutual_fund_code)
# Next, create a function that computes the cumulative returns and CAGR. We are going to do this for 1 year, 2 years, and 3 years.

def get_cumulative_returns(df,
                         nav_col='CLOSE',
                         date_col='date',
                         starting_date='1-1-2019',
                         ending_date ='31-12-2023'):
    start_date = pd.to_datetime(starting_date, format='%d-%m-%Y')
    end_date = pd.to_datetime(ending_date, format='%d-%m-%Y')

    df = (df\
          .sort_values(date_col)\
          .query(f"{date_col} >= @start_date and {date_col} <=@end_date")\
          .assign(daily_returns=lambda x: x[nav_col].pct_change(),
                  cumulative_daily_returns=lambda x: (x['daily_returns'] + 1).cumprod())\
          .reset_index(drop=True)\
         )

    return df

# We now pass our data — mutual funds and benchmark index — through the function to calculate metrics.

time_periods = {
    1: '1-1-2023',
    2: '1-1-2022',
    3: '1-1-2021'
}
results = []
time_period_dataframes = {time_period: dict() for time_period in time_periods}

# Index data
nifty_midcap_150 = 'NIFTY MIDCAP 150 INDEX'
for time_period, starting_date in time_periods.items():
    n_years = time_period
    nifty_midcap_cumulative = get_cumulative_returns(nifty_midcap_150,
                                                           nav_col='CLOSE',
                                                           date_col='date',
                                                           starting_date=starting_date,
                                                           ending_date ='31-12-2023')
    nifty_midcap_cumulative = nifty_midcap_cumulative.rename(columns={'Date':'date'})
    absolute_returns_prcnt = (nifty_midcap_cumulative['cumulative_daily_returns'].values[-1] - 1) * 100
    cagr = ((nifty_midcap_cumulative['CLOSE'].iloc[-1]/nifty_midcap_cumulative['CLOSE'].iloc[0]) ** (1/n_years) - 1) * 100

    time_period_dataframes[time_period][nifty_midcap_150] = nifty_midcap_cumulative
    results += [(nifty_midcap_150, time_period, absolute_returns_prcnt, cagr)]


# Mutual Funds data
mutual_funds_subset = ['PGIM India Midcap Opportunities Fund - Direct Plan - Growth Option',
'HDFC Mid-Cap Opportunities Fund - Growth Option - Direct Plan',
'Motilal Oswal Midcap Fund-Regular Plan-Growth Option',
'UTI Mid Cap Fund-Growth Option- Direct'
]

for instrument_desc in mutual_funds_subset:
    for time_period, starting_date in time_periods.items():
        n_years = time_period
        mutual_fund_data = mf_data_dict[instrument_desc].copy(deep=True)
        mf_with_cumulative = get_cumulative_returns(mutual_fund_data,
                                                           nav_col='nav',
                                                           date_col='date',
                                                           starting_date=starting_date,
                                                           ending_date ='31-12-2023')
        absolute_returns_prcnt = (mf_with_cumulative['cumulative_daily_returns'].values[-1] - 1) * 100
        cagr = ((mf_with_cumulative['nav'].iloc[-1]/mf_with_cumulative['nav'].iloc[0]) ** (1/n_years) - 1) * 100

        time_period_dataframes[time_period][instrument_desc] = mf_with_cumulative
        results += [(instrument_desc, time_period, absolute_returns_prcnt, cagr)]

# The results will be converted into a data frame.

result_df = pd.DataFrame(results, columns=['Fund', 'Years', 'Abs_returns_prcnt', 'CAGR']).\
pivot(index='Fund', columns='Years', values=['Abs_returns_prcnt', 'CAGR'])
result_df.columns = [f'{x}_{y}' for x, y in result_df.columns]
result_df = result_df.applymap(lambda x: f"{x:.2f}%")

time_period = 1

for idx, (instrument_desc, instrument_data) in enumerate(time_period_dataframes[time_period].items()):
    if idx == 0:
        ax = instrument_data.plot(y='cumulative_daily_returns', x='date', label=instrument_desc)
    else:
        instrument_data.plot(ax = ax, y='cumulative_daily_returns', x='date', label=instrument_desc)

plt.title(f"Cumulative Returns - {time_period}-year comparison")
ax.legend(loc='upper left')
plt.plot()


PGIM India Midcap Opportunities Fund - Direct Plan - Growth Option
HDFC Mid-Cap Opportunities Fund - Growth Option - Direct Plan
Motilal Oswal Midcap Fund-Regular Plan-Growth Option
UTI Mid Cap Fund-Growth Option- Direct


AttributeError: 'str' object has no attribute 'sort_values'